In [46]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [47]:
company_name = "GSB - UHE-ITA"

In [48]:
company = Company.objects.get(name=company_name)

In [49]:
from tqdm.notebook import tqdm
from openpyxl import load_workbook
from datetime import datetime
import json
from helpers.strings import get_obj_from_path
from helpers.signals import DisableSignals

In [50]:
file_name = "Cadastro de Instrumentos - UHIT-12-09"

In [51]:
def get_sheet_names(filename):
    wb = load_workbook(filename + ".xlsx")
    return wb.sheetnames

In [52]:
def load_sheet(filename, sheetname):
    
    wb = load_workbook(filename + ".xlsx", data_only=True)
    ws = wb[sheetname]
    
    header = []
    values = []

    for index, row in enumerate(ws.rows):
        if index == 0:
            continue
        elif index == 1:
            header = list([a.value for a in row])
            continue
        obj = {}
        for col_index, cell in enumerate(row):
            value = cell.value
            obj[header[col_index]] = value
        values.append(obj)
        
    return values

In [53]:
operational_control = OperationalControl.objects.get(
    firm__company=company,
    kind="701"
)

In [54]:
status = ServiceOrderActionStatus.objects.get(companies=company, name="Validado")
status

<ServiceOrderActionStatus: Validado - OCCURRENCE_RECORD_STATUS>

In [37]:
# for form in OccurrenceType.objects.filter(company=company, occurrence_kind="701"):
#     position = next(a for a in form.form_fields['fields'] if a['apiName'] == "operationalPosition")
#     position['selectOptions']['options'].append({"name": "Dique 1","value": "9"})
#     position['selectOptions']['options'].append({"name": "Dique 2","value": "10"})
#     position['selectOptions']['options'].append({"name": "Dique 3","value": "11"})
#     form.save()

In [55]:
firm = Firm.objects.get(company=company, name="Suporte")

In [56]:
sheets = get_sheet_names(file_name)
recs = []

for sheet in sheets:
    form = OccurrenceType.objects.filter(company=company, name=sheet)
    print("running", sheet)
    
    if not form.exists():
        print("skipping", sheet)
        continue
    
    form = form.first()

    values = load_sheet(file_name, sheet)

    for record in values:
        form_data = {}
        for key, value in record.items():
            if value is None or value == "":
                continue
                
            try:    
                field = next(a for a in form.form_fields['fields'] if a['displayName'].lower() == key.lower())
            except Exception as e:
                print("not found", key)
                continue
                
            
            if field['dataType'] in ["string", "textArea"]:
                form_data[field['apiName']] = str(value)
            elif field['dataType'] in ["float"]:
                form_data[field['apiName']] = float(value)
            elif field['dataType'] in ["select"]:
                options = get_obj_from_path(field, "selectoptions__options")
                
                if value == "Dique 1":
                    value = "Barragem Auxiliar 1"
                elif value == "Dique 2":
                    value = "Barragem Auxiliar 2"
                elif value == "Dique 3":
                    value = "Barragem Auxiliar 3"
                
                try:
                    choice = next(a['value'] for a in options if a['name'] == value)
                    form_data[field['apiName']] = choice
                except Exception as e:
                    print(key, value)
            elif field['dataType'] in ["boolean"]:
                if value.lower() == "sim":
                    form_data[field['apiName']] = True
                else:
                    form_data[field['apiName']] = False

        rec = OccurrenceRecord(
            company = company,
            occurrence_type = form,
            status = status,
            form_data = form_data,
            firm = firm,
            operational_control = operational_control,
            datetime = datetime(2023, 9, 11, 10, 0)
        )
        recs.append(rec)

running Instrumentos e posições
skipping Instrumentos e posições
running Medidor de Abertura de Junta
running Medidor de Nível d'Água
running Medidor de Vazão
running Medidor Triortogonal Mecânico
running Piezômetro Casagrande


In [57]:
OccurrenceRecord.objects.bulk_create(recs)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2023-09-11 10:00:00) while time zone support is active.
  RuntimeWarning)


[<OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:00:00>,
 <OccurrenceRecord: [GSB - UHE-ITA] -  - 2023-09-11 10:

### !!!!! Apagar tudo antes de importar?

In [ ]:
with DisableSignals():
    print(OccurrenceRecord.objects.filter(company=company).count())
    OccurrenceRecord.objects.filter(company=company).delete()